##Setting up imports (including dataset)

In [0]:
import sys
import tensorflow as tf
import numpy as np
import argparse
import os
import shutil
import pandas as pd
!pip install gspread
import gspread
import io
from sklearn.model_selection import train_test_split


#Package for neural networks
!pip install -q keras
import keras
from keras.models import Sequential
from keras.layers import Dense

from google.colab import files
uploaded = files.upload()

spambase = pd.read_csv(io.StringIO(uploaded['spambase.csv'].decode('utf-8')), header = None)
spambase

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple


##Initial Hypothesis

It doesn't look like there are hard and fast rules as to how many nodes should be in each layer, or how many layers there are, but a common starting point is an input layer with the number of features in each example, and the first hidden layer being half this amount. So our initial results will likely involve an input layer defined with 57 nodes to match the number of our features, and first hidden layer with 28 or so nodes. I think it might be better for our hidden layers to have a progressively smaller amount of nodes per level, which would mean that there will need to be aggregation of the results in previous layers. It might make sense then to go through the list of features and consider which ones may indicate an email is more likely to be spam and not spam (or neutral) and we can group our layers and nodes that way, and the result will be more effective.

##Creating and Running the Network (TensorFlow)

In [2]:
# Seperates independent and dependent variables
X = spambase.iloc[:,0:57].values
Y = spambase.iloc[:,57].values

# Uses sklearn module
# Creates training and test sets randomly with test set 20% of the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# Defining Architecture 
# Only one hidden layer (for now)
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer
  
# Defining amounts of nodes, number of classifications
n_hidden_1 = 28
n_input = X_train.shape[1]
n_classes = 2

# Sets up weights and biases (connects nodes)
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

keep_prob = tf.placeholder("float")

# 5000 training optimization runs, results every 1000, 32 rows loaded in each time
training_epochs = 5000
display_step = 1000
batch_size = 32

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder(tf.int64, [None])

predictions = multilayer_perceptron(x, weights, biases, keep_prob)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=predictions))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(X_train) / batch_size)
        x_batches = np.array_split(X_train, total_batch)
        y_batches = np.array_split(Y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: Y_test, keep_prob: 1.0}))

Epoch: 0001 cost= 901.615591033
Epoch: 1001 cost= 0.250808185
Epoch: 2001 cost= 0.158032463
Epoch: 3001 cost= 0.130299262
Epoch: 4001 cost= 0.127487279
Optimization Finished!
Accuracy: 0.95114005


##Creating and Running the Network (Keras)

In [6]:
# Initializing the Neural Network
spamClassifier = Sequential()

# Defining input layer and first hidden layer
# Number of nodes matches features, features/2
spamClassifier.add(Dense(units = 28, kernel_initializer = 'uniform', activation = 'relu', input_dim = 57))

# Defining 2nd layer
spamClassifier.add(Dense(units = 28, kernel_initializer = 'uniform', activation = 'relu'))

# Defining output layer
spamClassifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Setting hyperparameters
spamClassifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

spamClassifier.fit(X_train, Y_train, batch_size = 20, epochs = 100)

Epoch 1/100
3680/3680 [==============================] - 1s 172us/step - loss: 0.6001 - acc: 0.7198
Epoch 2/100
3680/3680 [==============================] - 0s 100us/step - loss: 0.4239 - acc: 0.8332
Epoch 3/100
3680/3680 [==============================] - 0s 102us/step - loss: 0.3114 - acc: 0.8856
Epoch 4/100
3680/3680 [==============================] - 0s 95us/step - loss: 0.2831 - acc: 0.8978
Epoch 5/100
3680/3680 [==============================] - 0s 91us/step - loss: 0.2435 - acc: 0.9120
Epoch 6/100
3680/3680 [==============================] - 0s 88us/step - loss: 0.2548 - acc: 0.9111
Epoch 7/100
3680/3680 [==============================] - 0s 90us/step - loss: 0.2460 - acc: 0.9122
Epoch 8/100
3680/3680 [==============================] - 0s 82us/step - loss: 0.2148 - acc: 0.9223
Epoch 9/100
3680/3680 [==============================] - 0s 81us/step - loss: 0.2280 - acc: 0.9201
Epoch 10/100
3680/3680 [==============================] - 0s 88us/step - loss: 0.2052 - acc: 0.9277
Epoch 

3680/3680 [==============================] - 0s 80us/step - loss: 0.1960 - acc: 0.9272
Epoch 17/100
3680/3680 [==============================] - 0s 86us/step - loss: 0.1931 - acc: 0.9323
Epoch 18/100
3680/3680 [==============================] - 0s 83us/step - loss: 0.1978 - acc: 0.9245
Epoch 19/100
3680/3680 [==============================] - 0s 84us/step - loss: 0.2162 - acc: 0.9223
Epoch 20/100
3680/3680 [==============================] - 0s 84us/step - loss: 0.2004 - acc: 0.9266
Epoch 21/100
3680/3680 [==============================] - 0s 84us/step - loss: 0.1816 - acc: 0.9340
Epoch 22/100
3680/3680 [==============================] - 0s 83us/step - loss: 0.1857 - acc: 0.9361
Epoch 23/100
3680/3680 [==============================] - 0s 89us/step - loss: 0.1739 - acc: 0.9380
Epoch 24/100
3680/3680 [==============================] - 0s 90us/step - loss: 0.2103 - acc: 0.9247
Epoch 25/100
3680/3680 [==============================] - 0s 88us/step - loss: 0.1927 - acc: 0.9310
Epoch 26/100


3680/3680 [==============================] - 0s 88us/step - loss: 0.1736 - acc: 0.9353
Epoch 32/100
3680/3680 [==============================] - 0s 83us/step - loss: 0.1778 - acc: 0.9345
Epoch 33/100
3680/3680 [==============================] - 0s 84us/step - loss: 0.1865 - acc: 0.9304
Epoch 34/100
3680/3680 [==============================] - 0s 90us/step - loss: 0.1819 - acc: 0.9342
Epoch 35/100
3680/3680 [==============================] - 0s 83us/step - loss: 0.1757 - acc: 0.9386
Epoch 36/100
3680/3680 [==============================] - 0s 90us/step - loss: 0.1746 - acc: 0.9342
Epoch 37/100
3680/3680 [==============================] - 0s 92us/step - loss: 0.1695 - acc: 0.9372
Epoch 38/100
3680/3680 [==============================] - 0s 90us/step - loss: 0.1641 - acc: 0.9394
Epoch 39/100
3680/3680 [==============================] - 0s 81us/step - loss: 0.1570 - acc: 0.9446
Epoch 40/100
3680/3680 [==============================] - 0s 88us/step - loss: 0.1850 - acc: 0.9364
Epoch 41/100


3680/3680 [==============================] - 0s 82us/step - loss: 0.1715 - acc: 0.9375
Epoch 47/100
3680/3680 [==============================] - 0s 81us/step - loss: 0.1752 - acc: 0.9361
Epoch 48/100
3680/3680 [==============================] - 0s 82us/step - loss: 0.1551 - acc: 0.9418
Epoch 49/100
3680/3680 [==============================] - 0s 87us/step - loss: 0.1472 - acc: 0.9470
Epoch 50/100
3680/3680 [==============================] - 0s 85us/step - loss: 0.1450 - acc: 0.9467
Epoch 51/100
3680/3680 [==============================] - 0s 80us/step - loss: 0.1492 - acc: 0.9446
Epoch 52/100
3680/3680 [==============================] - 0s 82us/step - loss: 0.1381 - acc: 0.9492
Epoch 53/100
3680/3680 [==============================] - 0s 87us/step - loss: 0.1461 - acc: 0.9470
Epoch 54/100
3680/3680 [==============================] - 0s 86us/step - loss: 0.1354 - acc: 0.9505
Epoch 55/100
3680/3680 [==============================] - 0s 82us/step - loss: 0.1504 - acc: 0.9424
Epoch 56/100


Epoch 62/100
3680/3680 [==============================] - 0s 83us/step - loss: 0.1408 - acc: 0.9481
Epoch 63/100
3680/3680 [==============================] - 0s 84us/step - loss: 0.1387 - acc: 0.9497
Epoch 64/100
3680/3680 [==============================] - 0s 82us/step - loss: 0.1311 - acc: 0.9511
Epoch 65/100
3680/3680 [==============================] - 0s 83us/step - loss: 0.1211 - acc: 0.9527
Epoch 66/100
3680/3680 [==============================] - 0s 82us/step - loss: 0.1328 - acc: 0.9465
Epoch 67/100
3680/3680 [==============================] - 0s 88us/step - loss: 0.1435 - acc: 0.9462
Epoch 68/100
3680/3680 [==============================] - 0s 81us/step - loss: 0.1496 - acc: 0.9467
Epoch 69/100
3680/3680 [==============================] - 0s 88us/step - loss: 0.1255 - acc: 0.9535
Epoch 70/100
3680/3680 [==============================] - 0s 82us/step - loss: 0.1304 - acc: 0.9522
Epoch 71/100
3680/3680 [==============================] - 0s 81us/step - loss: 0.1435 - acc: 0.9484


3680/3680 [==============================] - 0s 88us/step - loss: 0.1146 - acc: 0.9592
Epoch 78/100
3680/3680 [==============================] - 0s 86us/step - loss: 0.1108 - acc: 0.9576
Epoch 79/100
3680/3680 [==============================] - 0s 86us/step - loss: 0.1133 - acc: 0.9571
Epoch 80/100
3680/3680 [==============================] - 0s 87us/step - loss: 0.1162 - acc: 0.9560
Epoch 81/100
3680/3680 [==============================] - 0s 83us/step - loss: 0.1087 - acc: 0.9622
Epoch 82/100
3680/3680 [==============================] - 0s 84us/step - loss: 0.1071 - acc: 0.9611
Epoch 83/100
3680/3680 [==============================] - 0s 82us/step - loss: 0.1143 - acc: 0.9587
Epoch 84/100
3680/3680 [==============================] - 0s 84us/step - loss: 0.1122 - acc: 0.9576
Epoch 85/100
3680/3680 [==============================] - 0s 89us/step - loss: 0.1434 - acc: 0.9462
Epoch 86/100
3680/3680 [==============================] - 0s 83us/step - loss: 0.1228 - acc: 0.9527
Epoch 87/100


3680/3680 [==============================] - 0s 79us/step - loss: 0.1073 - acc: 0.9603
Epoch 93/100
3680/3680 [==============================] - 0s 82us/step - loss: 0.1072 - acc: 0.9603
Epoch 94/100
3680/3680 [==============================] - 0s 82us/step - loss: 0.1056 - acc: 0.9614
Epoch 95/100
3680/3680 [==============================] - 0s 89us/step - loss: 0.1050 - acc: 0.9630
Epoch 96/100
3680/3680 [==============================] - 0s 82us/step - loss: 0.1049 - acc: 0.9603
Epoch 97/100
3680/3680 [==============================] - 0s 89us/step - loss: 0.0973 - acc: 0.9644
Epoch 98/100
3680/3680 [==============================] - 0s 84us/step - loss: 0.0970 - acc: 0.9636
Epoch 99/100
3680/3680 [==============================] - 0s 88us/step - loss: 0.0993 - acc: 0.9625
Epoch 100/100
3680/3680 [==============================] - 0s 84us/step - loss: 0.1012 - acc: 0.9617


##Resources:</br>
https://towardsdatascience.com/neural-networks-with-google-colaboratory-artificial-intelligence-getting-started-713b5eb07f14 </br>
https://medium.com/@curiousily/tensorflow-for-hackers-part-ii-building-simple-neural-network-2d6779d2f91b</br>
https://www.analyticsvidhya.com/blog/2016/10/an-introduction-to-implementing-neural-networks-using-tensorflow/